# **Distribución % de créditos por organismo**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "financiamientos" in nombre and "pachuca" in nombre
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2019_pachuca',
 'df_financiamientos_2020_pachuca',
 'df_financiamientos_2021_pachuca',
 'df_financiamientos_2022_pachuca',
 'df_financiamientos_2023_pachuca',
 'df_financiamientos_2024_pachuca']

## 2022

In [2]:
finance = df_financiamientos_2022_pachuca[['id','organismo']]
finance

,id,organismo
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1
...,...,...
1383,1384,4
1384,1385,4
1385,1386,4
1386,1387,4


In [3]:
# Convertir la columna 'organismo' a cadena de texto (str)
#finance['organismo'] = finance['organismo'].astype(str)
finance.loc[:, 'organismo'] = finance['organismo'].astype(str)

# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    '1': 'INFONAVIT',
    '2': 'CNBV',
    '3': 'FOVISSTE',
    '5': 'CONAVI',
    '7': 'BANJERCITO',
    '26': 'INSUS'
}
# Reemplazar los valores de 'organismo' con el texto correspondiente utilizando replace
#finance['organismo'] = finance['organismo'].replace(categorias)
finance.loc[:, 'organismo'] = finance['organismo'].replace(categorias)

# Reemplazar cualquier otro número que no esté en el diccionario con '-'
#finance['organismo'] = finance['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)
finance.loc[:, 'organismo'] = finance['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)

# Eliminar la fila donde rango_ingresos es '0'
finance = finance[finance['organismo'] != '-']

# Obtener el conteo de registros por cada categoría en la columna 'organismo'
conteo_organismos = finance['organismo'].value_counts().reset_index()
conteo_organismos.columns = ['organismo', 'conteo']


# Asegurarse de que todas las categorías estén presentes en el conteo, incluso si tienen un conteo de 0
for categoria in categorias.values():
    if categoria not in conteo_organismos['organismo'].unique():
        nuevo_registro = pd.DataFrame({'organismo': [categoria], 'conteo': [0]})
        conteo_organismos = pd.concat([conteo_organismos, nuevo_registro], ignore_index=True)

# Ordenar el DataFrame por las categorías en orden ascendente
conteo_organismos = conteo_organismos.sort_values(by='organismo').reset_index(drop=True)

print(conteo_organismos)

    organismo  conteo
0  BANJERCITO       7
1        CNBV     578
2      CONAVI       0
3    FOVISSTE     291
4   INFONAVIT     471
5       INSUS       0


C:\Users\yoe11\AppData\Local\Temp\ipykernel_4292\2664051192.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1' '1' '1' ... '4' '4' '4']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  finance.loc[:, 'organismo'] = finance['organismo'].astype(str)


In [4]:
# Crear un DataFrame con el conteo de registros por cada organismo
df_conteo_organismos = finance['organismo'].value_counts().reset_index()
df_conteo_organismos.columns = ['organismo', 'conteo']

# Calcular el total de registros en el DataFrame original
total_registros = len(finance)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
df_conteo_organismos['promedio'] = round((df_conteo_organismos['conteo'] * 100) / total_registros,2)
df_conteo_organismos

,organismo,conteo,promedio
0,CNBV,578,42.91
1,INFONAVIT,471,34.97
2,FOVISSTE,291,21.60
3,BANJERCITO,7,0.52


In [5]:
# Calcular el total de registros en el DataFrame original
total_registros = len(finance)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
conteo_organismos['promedio'] = round((conteo_organismos['conteo'] * 100) / total_registros,2)
conteo_organismos

,organismo,conteo,promedio
0,BANJERCITO,7,0.52
1,CNBV,578,42.91
2,CONAVI,0,0.00
3,FOVISSTE,291,21.60
4,INFONAVIT,471,34.97
5,INSUS,0,0.00


In [6]:
f_2022 = finance.size
cont = df_conteo_organismos['conteo'].sum()
print("Hay ", f_2022, " registros en la base")
print("Hay ", cont, " registros al contar")

Hay  2694  registros en la base
Hay  1347  registros al contar


In [7]:
# Definir el diccionario de orden de organismos
orden_organismos = {
    'INFONAVIT': 1,
    'CNBV': 2,
    'FOVISSTE': 3,
    'BANJERCITO': 4
}

#df_conteo_organismos['orden'] = df_conteo_organismos['organismo'].map(orden_organismos)
df_conteo_organismos.loc[:, 'orden'] = df_conteo_organismos['organismo'].map(orden_organismos)

# Ordenar el DataFrame por el orden de los organismos
df_conteo_organismos = df_conteo_organismos.sort_values(by='orden')
df_conteo_organismos = df_conteo_organismos.drop(columns=['orden'])
df_conteo_organismos

,organismo,conteo,promedio
1,INFONAVIT,471,34.97
0,CNBV,578,42.91
2,FOVISSTE,291,21.60
3,BANJERCITO,7,0.52


In [10]:

labels = df_conteo_organismos['organismo']
values = df_conteo_organismos['promedio']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.2f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos por organismo (2022)',
    #title_font_size = 22,
    xaxis=dict(title="Organismo"),
    yaxis=dict(
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

guardar_grafico_como_html(fig, 'g_bar_creditos_organismo_2022', carpeta='assets\graficas')
fig.show()

<>:31: SyntaxWarning:

invalid escape sequence '\g'

<>:47: SyntaxWarning:

invalid escape sequence '\g'

<>:31: SyntaxWarning:

invalid escape sequence '\g'

<>:47: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_4292\1935841576.py:31: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_4292\1935841576.py:47: SyntaxWarning:

invalid escape sequence '\g'



## 2023

In [11]:
finance = df_financiamientos_2023_pachuca[['id','organismo']]
finance

,id,organismo
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1
...,...,...
1394,1395,3
1395,1396,3
1396,1397,3
1397,1398,4


In [12]:
# Convertir la columna 'organismo' a cadena de texto (str)
#finance['organismo'] = finance['organismo'].astype(str)
finance.loc[:, 'organismo'] = finance['organismo'].astype(str)

# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    '1': 'INFONAVIT',
    '2': 'CNBV',
    '3': 'FOVISSTE',
    '5': 'CONAVI',
    '7': 'BANJERCITO',
    '26': 'INSUS'
}
# Reemplazar los valores de 'organismo' con el texto correspondiente utilizando replace
#finance['organismo'] = finance['organismo'].replace(categorias)
finance.loc[:, 'organismo'] = finance['organismo'].replace(categorias)

# Reemplazar cualquier otro número que no esté en el diccionario con '-'
#finance['organismo'] = finance['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)
finance.loc[:, 'organismo'] = finance['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)

# Eliminar la fila donde rango_ingresos es '0'
finance = finance[finance['organismo'] != '-']

# Obtener el conteo de registros por cada categoría en la columna 'organismo'
conteo_organismos = finance['organismo'].value_counts().reset_index()
conteo_organismos.columns = ['organismo', 'conteo']


# Asegurarse de que todas las categorías estén presentes en el conteo, incluso si tienen un conteo de 0
for categoria in categorias.values():
    if categoria not in conteo_organismos['organismo'].unique():
        nuevo_registro = pd.DataFrame({'organismo': [categoria], 'conteo': [0]})
        conteo_organismos = pd.concat([conteo_organismos, nuevo_registro], ignore_index=True)

# Ordenar el DataFrame por las categorías en orden ascendente
conteo_organismos = conteo_organismos.sort_values(by='organismo').reset_index(drop=True)

print(conteo_organismos)

    organismo  conteo
0  BANJERCITO       3
1        CNBV     475
2      CONAVI       6
3    FOVISSTE     252
4   INFONAVIT     627
5       INSUS      17


C:\Users\yoe11\AppData\Local\Temp\ipykernel_4292\2664051192.py:3: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1' '1' '1' ... '3' '4' '4']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



In [13]:
# Crear un DataFrame con el conteo de registros por cada organismo
df_conteo_organismos = finance['organismo'].value_counts().reset_index()
df_conteo_organismos.columns = ['organismo', 'conteo']

# Calcular el total de registros en el DataFrame original
total_registros = len(finance)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
df_conteo_organismos['promedio'] = round((df_conteo_organismos['conteo'] * 100) / total_registros,2)
df_conteo_organismos

,organismo,conteo,promedio
0,INFONAVIT,627,45.43
1,CNBV,475,34.42
2,FOVISSTE,252,18.26
3,INSUS,17,1.23
4,CONAVI,6,0.43
5,BANJERCITO,3,0.22


In [14]:
# Calcular el total de registros en el DataFrame original
total_registros = len(finance)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
conteo_organismos['promedio'] = round((conteo_organismos['conteo'] * 100) / total_registros,2)
conteo_organismos

,organismo,conteo,promedio
0,BANJERCITO,3,0.22
1,CNBV,475,34.42
2,CONAVI,6,0.43
3,FOVISSTE,252,18.26
4,INFONAVIT,627,45.43
5,INSUS,17,1.23


In [15]:
f_2022 = finance.size
cont = df_conteo_organismos['conteo'].sum()
print("Hay ", f_2022, " registros en la base")
print("Hay ", cont, " registros al contar")

Hay  2760  registros en la base
Hay  1380  registros al contar


In [16]:
# Definir el diccionario de orden de organismos
orden_organismos = {
    'INFONAVIT': 1,
    'CNBV': 2,
    'FOVISSTE': 3,
    'BANJERCITO': 4
}

#df_conteo_organismos['orden'] = df_conteo_organismos['organismo'].map(orden_organismos)
df_conteo_organismos.loc[:, 'orden'] = df_conteo_organismos['organismo'].map(orden_organismos)

# Ordenar el DataFrame por el orden de los organismos
df_conteo_organismos = df_conteo_organismos.sort_values(by='orden')
df_conteo_organismos = df_conteo_organismos.drop(columns=['orden'])
df_conteo_organismos

,organismo,conteo,promedio
0,INFONAVIT,627,45.43
1,CNBV,475,34.42
2,FOVISSTE,252,18.26
5,BANJERCITO,3,0.22
3,INSUS,17,1.23
4,CONAVI,6,0.43


In [17]:

labels = df_conteo_organismos['organismo']
values = df_conteo_organismos['promedio']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.2f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos por organismo (2022)',
    #title_font_size = 22,
    xaxis=dict(title="Organismo"),
    yaxis=dict(
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

guardar_grafico_como_html(fig, 'g_bar_creditos_organismo_2023', carpeta='assets\graficas')
fig.show()

<>:31: SyntaxWarning:

invalid escape sequence '\g'

<>:47: SyntaxWarning:

invalid escape sequence '\g'

<>:31: SyntaxWarning:

invalid escape sequence '\g'

<>:47: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_4292\1892731859.py:31: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_4292\1892731859.py:47: SyntaxWarning:

invalid escape sequence '\g'



## 2024

In [18]:
finance = df_financiamientos_2024_pachuca[['id','organismo']]
finance

,id,organismo
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1
...,...,...
787,788,3
788,789,3
789,790,4
790,791,4


In [19]:
# Convertir la columna 'organismo' a cadena de texto (str)
#finance['organismo'] = finance['organismo'].astype(str)
finance.loc[:, 'organismo'] = finance['organismo'].astype(str)

# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    '1': 'INFONAVIT',
    '2': 'CNBV',
    '3': 'FOVISSTE',
    '5': 'CONAVI',
    '7': 'BANJERCITO',
    '26': 'INSUS'
}
# Reemplazar los valores de 'organismo' con el texto correspondiente utilizando replace
#finance['organismo'] = finance['organismo'].replace(categorias)
finance.loc[:, 'organismo'] = finance['organismo'].replace(categorias)

# Reemplazar cualquier otro número que no esté en el diccionario con '-'
#finance['organismo'] = finance['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)
finance.loc[:, 'organismo'] = finance['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)

# Eliminar la fila donde rango_ingresos es '0'
finance = finance[finance['organismo'] != '-']

# Obtener el conteo de registros por cada categoría en la columna 'organismo'
conteo_organismos = finance['organismo'].value_counts().reset_index()
conteo_organismos.columns = ['organismo', 'conteo']


# Asegurarse de que todas las categorías estén presentes en el conteo, incluso si tienen un conteo de 0
for categoria in categorias.values():
    if categoria not in conteo_organismos['organismo'].unique():
        nuevo_registro = pd.DataFrame({'organismo': [categoria], 'conteo': [0]})
        conteo_organismos = pd.concat([conteo_organismos, nuevo_registro], ignore_index=True)

# Ordenar el DataFrame por las categorías en orden ascendente
conteo_organismos = conteo_organismos.sort_values(by='organismo').reset_index(drop=True)

print(conteo_organismos)

    organismo  conteo
0  BANJERCITO       2
1        CNBV     259
2      CONAVI       0
3    FOVISSTE     134
4   INFONAVIT     376
5       INSUS       0


C:\Users\yoe11\AppData\Local\Temp\ipykernel_4292\2664051192.py:3: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '2' '2' '2' '2' '2'
 '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2'
 '2' '2' '3' '3' '3' '3' '3' '3' '3' '3' '3' '3' '3' '3' '3' '3' '4' '4'
 '4' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2'
 '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2'
 '2' '2' '2' '2' '2' '2' '3' '3' '3' '3' '3' '3' '3' '3' '3' '4' '1' '1'
 '1' '1' '1'

In [20]:
# Crear un DataFrame con el conteo de registros por cada organismo
df_conteo_organismos = finance['organismo'].value_counts().reset_index()
df_conteo_organismos.columns = ['organismo', 'conteo']

# Calcular el total de registros en el DataFrame original
total_registros = len(finance)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
df_conteo_organismos['promedio'] = round((df_conteo_organismos['conteo'] * 100) / total_registros,2)
df_conteo_organismos

,organismo,conteo,promedio
0,INFONAVIT,376,48.77
1,CNBV,259,33.59
2,FOVISSTE,134,17.38
3,BANJERCITO,2,0.26


In [21]:
# Calcular el total de registros en el DataFrame original
total_registros = len(finance)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
conteo_organismos['promedio'] = round((conteo_organismos['conteo'] * 100) / total_registros,2)
conteo_organismos

,organismo,conteo,promedio
0,BANJERCITO,2,0.26
1,CNBV,259,33.59
2,CONAVI,0,0.00
3,FOVISSTE,134,17.38
4,INFONAVIT,376,48.77
5,INSUS,0,0.00


In [22]:
f_2022 = finance.size
cont = df_conteo_organismos['conteo'].sum()
print("Hay ", f_2022, " registros en la base")
print("Hay ", cont, " registros al contar")

Hay  1542  registros en la base
Hay  771  registros al contar


In [23]:
# Definir el diccionario de orden de organismos
orden_organismos = {
    'INFONAVIT': 1,
    'CNBV': 2,
    'FOVISSTE': 3,
    'BANJERCITO': 4
}

#df_conteo_organismos['orden'] = df_conteo_organismos['organismo'].map(orden_organismos)
df_conteo_organismos.loc[:, 'orden'] = df_conteo_organismos['organismo'].map(orden_organismos)

# Ordenar el DataFrame por el orden de los organismos
df_conteo_organismos = df_conteo_organismos.sort_values(by='orden')
df_conteo_organismos = df_conteo_organismos.drop(columns=['orden'])
df_conteo_organismos

,organismo,conteo,promedio
0,INFONAVIT,376,48.77
1,CNBV,259,33.59
2,FOVISSTE,134,17.38
3,BANJERCITO,2,0.26


In [24]:

labels = df_conteo_organismos['organismo']
values = df_conteo_organismos['promedio']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.2f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos por organismo (2022)',
    #title_font_size = 22,
    xaxis=dict(title="Organismo"),
    yaxis=dict(
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

guardar_grafico_como_html(fig, 'g_bar_creditos_organismo_2024', carpeta='assets\graficas')
fig.show()

<>:31: SyntaxWarning:

invalid escape sequence '\g'

<>:47: SyntaxWarning:

invalid escape sequence '\g'

<>:31: SyntaxWarning:

invalid escape sequence '\g'

<>:47: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_4292\2492648266.py:31: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_4292\2492648266.py:47: SyntaxWarning:

invalid escape sequence '\g'

